<a href="https://colab.research.google.com/github/sanjshine99/6CS005-High-Performance-Computing/blob/main/HPCTask3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Password Cracking using CUDA

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp5s8n4dqj".


In [ ]:
!nvidia-smi

Fri May 24 09:20:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

__device__ void crypt(char *rawPassword, char *newPassword)
{
	newPassword[0] = rawPassword[0] + 2;
	newPassword[1] = rawPassword[0] - 2;
	newPassword[2] = rawPassword[0] + 1;
	newPassword[3] = rawPassword[1] + 3;
	newPassword[4] = rawPassword[1] - 3;
	newPassword[5] = rawPassword[1] - 1;
	newPassword[6] = rawPassword[2] + 2;
	newPassword[7] = rawPassword[2] - 2;
	newPassword[8] = rawPassword[3] + 4;
	newPassword[9] = rawPassword[3] - 4;
	newPassword[10] = '\0';

	for (int i = 0; i < 10; i++)
	{
		if (i < 6)
		{ // for letters
			if (newPassword[i] > 122)
			{
				newPassword[i] = (newPassword[i] - 122) + 97;
			}
			else if (newPassword[i] < 97)
			{
				newPassword[i] = (97 - newPassword[i]) + 97;
			}
		}
		else
		{ // for numbers
			if (newPassword[i] > 57)
			{
				newPassword[i] = (newPassword[i] - 57) + 48;
			}
			else if (newPassword[i] < 48)
			{
				newPassword[i] = (48 - newPassword[i]) + 48;
			}
		}
	}
}

// Global function to be executed on GPU
__global__ void encryptKernel(char *rawPasswords, char *encryptedPasswords, int passwordCount)
{
	int idx = blockIdx.x * blockDim.x + threadIdx.x;
	if (idx < passwordCount)
	{
		crypt(&rawPasswords[idx * 4], &encryptedPasswords[idx * 11]);
	}
}

int main()
{
	char *rawPasswords = "ag13"; // Example password
	if (strlen(rawPasswords) != 4 ||
		!(rawPasswords[0] >= 'a' && rawPasswords[0] <= 'z') ||
		!(rawPasswords[1] >= 'a' && rawPasswords[1] <= 'z') ||
		!(rawPasswords[2] >= '0' && rawPasswords[2] <= '9') ||
		!(rawPasswords[3] >= '0' && rawPasswords[3] <= '9'))
	{
		printf("Error: Password must be in the format 'LetterLetterNumberNumber' (e.g., 'ab12') and contain only lowercase letters and numbers.\n");
		return -1;
	}
	else
	{
		char *d_rawPasswords, *d_encryptedPasswords;
		char encryptedPasswords[11];

		// Allocate memory on device
		cudaMalloc((void **)&d_rawPasswords, 4 * sizeof(char));
		cudaMalloc((void **)&d_encryptedPasswords, 11 * sizeof(char));

		cudaError_t err;

		// Allocate memory on device
		err = cudaMalloc((void **)&d_rawPasswords, 4 * sizeof(char));
		if (err != cudaSuccess)
		{
			printf("CUDA malloc failed for d_rawPasswords: %s\n", cudaGetErrorString(err));
			return -1; // or handle error appropriately
		}

		err = cudaMalloc((void **)&d_encryptedPasswords, 11 * sizeof(char));
		if (err != cudaSuccess)
		{
			printf("CUDA malloc failed for d_encryptedPasswords: %s\n", cudaGetErrorString(err));
			cudaFree(d_rawPasswords); // free already allocated resources
			return -1;				  // or handle error appropriately
		}

		// Copy data from host to device
		err = cudaMemcpy(d_rawPasswords, rawPasswords, 4 * sizeof(char), cudaMemcpyHostToDevice);
		if (err != cudaSuccess)
		{
			printf("CUDA memcpy failed from host to device: %s\n", cudaGetErrorString(err));
			cudaFree(d_rawPasswords);
			cudaFree(d_encryptedPasswords);
			return -1;
		}

		// Launch kernel with one block and one thread per block (as example)
		encryptKernel<<<1, 1>>>(d_rawPasswords, d_encryptedPasswords, 1);
		err = cudaDeviceSynchronize();
		if (err != cudaSuccess)
		{
			printf("CUDA kernel launch failed: %s\n", cudaGetErrorString(err));
			cudaFree(d_rawPasswords);
			cudaFree(d_encryptedPasswords);
			return -1;
		}

		// Copy result back to host
		err = cudaMemcpy(encryptedPasswords, d_encryptedPasswords, 11 * sizeof(char), cudaMemcpyDeviceToHost);
		if (err != cudaSuccess)
		{
			printf("CUDA memcpy failed from device to host: %s\n", cudaGetErrorString(err));
			cudaFree(d_rawPasswords);
			cudaFree(d_encryptedPasswords);
			return -1;
		}

		// Copy data from host to device
		cudaMemcpy(d_rawPasswords, rawPasswords, 4 * sizeof(char), cudaMemcpyHostToDevice);

		// Launch kernel with one block and one thread per block (as example)
		encryptKernel<<<1, 1>>>(d_rawPasswords, d_encryptedPasswords, 1);

		// Copy result back to host
		cudaMemcpy(encryptedPasswords, d_encryptedPasswords, 11 * sizeof(char), cudaMemcpyDeviceToHost);

		// Print encrypted password
		printf("Encrypted Password: %s\n", encryptedPasswords);

		// Free memory
		cudaFree(d_rawPasswords);
		cudaFree(d_encryptedPasswords);

		return 0;
	}
}

UsageError: Cell magic `%%cuda` not found.


In [ ]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

__device__
char* CudaCrypt(char* rawPassword) {

	char * newPassword = (char *) malloc(sizeof(char) * 11);

	newPassword[0] = rawPassword[0] + 2;
	newPassword[1] = rawPassword[0] - 2;
	newPassword[2] = rawPassword[0] + 1;
	newPassword[3] = rawPassword[1] + 3;
	newPassword[4] = rawPassword[1] - 3;
	newPassword[5] = rawPassword[1] - 1;
	newPassword[6] = rawPassword[2] + 2;
	newPassword[7] = rawPassword[2] - 2;
	newPassword[8] = rawPassword[3] + 4;
	newPassword[9] = rawPassword[3] - 4;
	newPassword[10] = '\0';

	for(int i =0; i<10; i++){
		if(i >= 0 && i < 6){ //checking letter limits
			if(newPassword[i] > 122){
				newPassword[i] = (newPassword[i] - 122) + 97;
			}else if(newPassword[i] < 97){
				newPassword[i] = (97 - newPassword[i]) + 97;
			}
		}else{ //checking numbers
			if(newPassword[i] > 57){
				newPassword[i] = (newPassword[i] - 57) + 48;
			}else if(newPassword[i] < 48){
				newPassword[i] = (48 - newPassword[i]) + 48;
			}
		}
	}
	return newPassword;
}

__device__
int isEncryptedMatching(char* one, char* two, int length) {
	int result = 1;
	for (int i = 0; i < length; i++) {
		if (one[i] != two[i]) {
			result = 0;
			break;
		}
	}
	return result;
}

__global__
void decryptPass(char* alphabet, char* numbers, char* encryptedPassword, char* deviceOutputPass){
	/// Get cuda thread id
	int uid = blockDim.x * blockIdx.x + threadIdx.x;

	if(*deviceOutputPass != NULL) {

		return;
	}

	/// Create password to check on this thread
	char potentialPassword[4];
	potentialPassword[0] = alphabet[blockIdx.x];
	potentialPassword[1] = alphabet[blockIdx.y];
	potentialPassword[2] = numbers[threadIdx.x];
	potentialPassword[3] = numbers[threadIdx.y];

	/// Encrypt the password
	char* encryptedPotential;
	encryptedPotential = CudaCrypt(potentialPassword);


	/// Check the current password matches to the target encryptedPassword
	if ( isEncryptedMatching(encryptedPassword, encryptedPotential, 11) > 0 )
	{

		printf("UID '%d' Encrypted password '%s' from combination '%s' matches potential password = '%s'\n", uid, encryptedPassword, potentialPassword, encryptedPotential);

		for (int i = 0; i < 4; i++ ) {
			deviceOutputPass[i] = potentialPassword[i];
		}
	}
}

int main(int argc, char** argv) {

	char* encryptedPassword = "dbcfbb2771";
	if (argc > 1) {
		encryptedPassword = argv[1];
	}

	printf("Password is: '%s'\n", encryptedPassword);

	// Initialize alpha numeric characters
	char cpuAlphabet[26] = { 'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z' };
	char cpuNumbers[10] = { '0', '1', '2', '3', '4', '5', '6' ,'7', '8', '9' };

	int sizeOfEncryptedPassword = sizeof(char) * 11;

	char* cudaAlphabet;
	cudaMalloc( (void**) &cudaAlphabet, sizeof(char) * 26 );
	cudaMemcpy( cudaAlphabet, cpuAlphabet, sizeof(char) * 26, cudaMemcpyHostToDevice );

	char* cudaNumbers;
	cudaMalloc( (void**) &cudaNumbers, sizeof(char) * 10 );
	cudaMemcpy( cudaNumbers, cpuNumbers, sizeof(char) * 10, cudaMemcpyHostToDevice );

	char* cudaEncryptedPassword;
	cudaMalloc( (void**) &cudaEncryptedPassword, sizeOfEncryptedPassword );
	cudaMemcpy( cudaEncryptedPassword, encryptedPassword, sizeOfEncryptedPassword, cudaMemcpyHostToDevice);

	char* cudaOutputPass;
	cudaMalloc( (void**) &cudaOutputPass, sizeOfEncryptedPassword );

	/// Launch cuda threads
	decryptPass<<< dim3(26, 26, 1), dim3(10, 10, 1) >>>(cudaAlphabet, cudaNumbers, cudaEncryptedPassword, cudaOutputPass);
	cudaDeviceSynchronize();

	printf("Finished executing CUDA threads\n");

	/// Send CUDA decrypted password to the CPU
	char* cpuOutputPass = (char*)malloc( sizeof(char) * 4 );
	cudaMemcpy(cpuOutputPass, cudaOutputPass, sizeOfEncryptedPassword, cudaMemcpyDeviceToHost);

	/// Print results
	printf("*********\n");
	printf("Results:\n");
	if (cpuOutputPass != NULL && cpuOutputPass[0] != 0) {
		printf("Encrypted Password: '%s'\n", encryptedPassword);
		printf("Decrypted Password: '%s'\n", cpuOutputPass);
	} else {
		printf("Password decryption aborted.\n");
	}

	cudaFree(cudaAlphabet);
	cudaFree(cudaNumbers);
	cudaFree(cudaEncryptedPassword);
	cudaFree(cudaOutputPass);
	free(cpuOutputPass);
}


Password is: 'dbcfbb2771'
UID '19' Encrypted password 'dbcfbb2771' from combination 'bc93' matches potential password = 'dbcfbb2771'
Finished executing CUDA threads
*********
Results:
Encrypted Password: 'dbcfbb2771'
Decrypted Password: 'bc93'

